In [1]:
from langchain_community.document_loaders import PyPDFLoader
import lancedb
from langchain_community.vectorstores import LanceDB
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [5]:
model = HuggingFaceEmbeddings(model_name = 'all-MiniLM-L6-v2')

In [11]:
path = [ 
      
        "../RAG/resume.pdf",
        "../RAG/resume copy.pdf"
    
]

In [13]:
docs = [PyPDFLoader(url).load() for url in path]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)


In [14]:
doc_splits

[Document(metadata={'source': '../RAG/resume.pdf', 'page': 0}, page_content="Prompt Engineering for LLMs\nGenerative AI Solutions (GPT, Claude,\nStable Diffusion, GANs, Langchain)\nAWS and GCP cloud services(Bedrock,\nSagemaker, Amazon Titan)\nMLOps and DevOps (AWS & GCP)\nDeep Learning (Tensorflow, Keras)\nArtificial Intelligence and Machine\nLearning\nComputer Vision (Pytorch)\nCognitive Learning (NLP)\nData Analytics and Visualization\n(PowerBI , Tableau)\nDatabase Management\nFront-end Development\nPython (Django)TECH SKILLSEDUCATION\n Jain (Deemed-to-be) University, Bangalore\n2020-2024\nSOFT SKILLS\nProblem-Solving\nCritical Thinking\nPublic Speaking\nTeam Management\nConflict Resolution\nCommunication\nProfessional presentation\nHackathon managementBachelor of Technology in Computer Science and Engineering \nspecializing in Artificial Intelligence and Machine Learning\nSGPA: 9.82/10\nEXPERIENCESri Chaitanya Techno School, Bangalore\nPhysics, Chemistry, Math, Biology11 th-12th - 

In [15]:
db = lancedb.connect("/tmp/lancedb")
table = db.create_table(
    "food_table",
    data=[{"vector": model.embed_query("Hello World"), "text": "Hello World"}],
    mode="overwrite",
)

vectorstore = LanceDB.from_documents(
    documents=doc_splits,
    embedding=model,
    connection=table,
)
retriever = vectorstore.as_retriever()

In [19]:
docs=retriever.invoke("what skills are good?")

In [20]:
type(docs)

list

In [25]:
text = [d.page_content for d in docs]

In [26]:
text

["Prompt Engineering for LLMs\nGenerative AI Solutions (GPT, Claude,\nStable Diffusion, GANs, Langchain)\nAWS and GCP cloud services(Bedrock,\nSagemaker, Amazon Titan)\nMLOps and DevOps (AWS & GCP)\nDeep Learning (Tensorflow, Keras)\nArtificial Intelligence and Machine\nLearning\nComputer Vision (Pytorch)\nCognitive Learning (NLP)\nData Analytics and Visualization\n(PowerBI , Tableau)\nDatabase Management\nFront-end Development\nPython (Django)TECH SKILLSEDUCATION\n Jain (Deemed-to-be) University, Bangalore\n2020-2024\nSOFT SKILLS\nProblem-Solving\nCritical Thinking\nPublic Speaking\nTeam Management\nConflict Resolution\nCommunication\nProfessional presentation\nHackathon managementBachelor of Technology in Computer Science and Engineering \nspecializing in Artificial Intelligence and Machine Learning\nSGPA: 9.82/10\nEXPERIENCESri Chaitanya Techno School, Bangalore\nPhysics, Chemistry, Math, Biology11 th-12th - CBSE\n2020\nPercentage: 86.4%\nBGS Public School, Bangalore\n  10th - CBSE\